# BuildIn Tool

### DuckDuckGo

In [3]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
result = search_tool.invoke("tell top news in india today")
print(result)

India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. India News, Latest News India: Read Live Breaking News from India along with Latest News and Today's Top Headlines Online in Bharat. Stay Up-to-date with Top news in India, current headlines, photos & videos online, live coverage on entertainment, business, politics, sports, technology and more at indianexpress.com. Bitter Bite: Chhattisgarh Naxal Operation. India Today covers a major operation in Chhattisgarh, neutralising 30 Naxals, including top commander Basavaraju with a 1-crore bounty.Who was he? India Today profiles the face of Naxals. Legal Platter: Courtroom Battles. India Today tracks the Supreme Court hearing the Waqf Amendment Act case Thursday, with the Centre arguing Waqf isn't essential ... Get updated with current top news stories from India and the world only on Zee News. Zee Ne

In [4]:
print(search_tool.name)
print(search_tool.description) #  Prints the description attribute of the search_tool object.
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### BuildIn Shell Tool

In [5]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('cd')

print(results)

Executing command:
 cd
c:\Users\shubu\Desktop\GenAI\14.Tools



c:\Users\shubu\Desktop\GenAI\venv\lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


# Custom Tools

### Method 1 - Using @tool Decorator

In [6]:
from langchain_core.tools import tool

In [7]:
# step 1 - create a function
def mul(a,b):
    '''mulitply two numbers''' # make sure to add doc. string for llm 
    return a*b

In [8]:
# step 2 - and type hints(not important but recommended)
def mul(a:int, b:int) -> int:
    '''mulitply two numbers'''
    return a*b


In [9]:
# step 3 - add tool decorator

@tool #this funtion make it a special function to communicate with LLM
def mul(a:int, b:int) -> int:
    '''mulitply two numbers'''
    return a*b

In [10]:
result = mul.invoke({"a":5, "b":8})
print(result)

40


In [11]:
print(mul.name)
print(mul.description)
print(mul.args)

mul
mulitply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [12]:
print(mul.args_schema.model_json_schema()) #tool given to llm but what does llm sees 
# paste in json formatter we send schema of tool to llm

{'description': 'mulitply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'mul', 'type': 'object'}


### Method 2 - Using Structured Tool

In [13]:
from langchain.tools import StructuredTool
from pydantic import Field, BaseModel

In [14]:
# pydantic model 
class MultiplyInput(BaseModel):
    a: int = Field(required = True, description="The first number to multiply")
    b: int = Field(required = True, description="The second number to multiply")

In [15]:
# function
def mul(a:int , b:int) -> int:
    return a*b

In [16]:
multiply_tool = StructuredTool.from_function(
    func = mul, # function
    name="multiply", # give a name 
    description="multiply two numbers", # give a description
    args_schema=MultiplyInput # sending our argument schema
)

In [17]:
result = multiply_tool.invoke({"a":5, "b":8})
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

40
multiply
multiply two numbers
{'a': {'description': 'The first number to multiply', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


### Method 3 - Base Tool

In [18]:
from langchain.tools import BaseTool
from typing import Type

In [19]:
#arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(required = True, description="The first number to multiply")
    b: int = Field(required = True, description="The second number to multiply")

In [ ]:
class MultiplyTool(BaseTool): # MultiplyTool class is the child of base tool class
    name : str = "multiply" # we can make our attribute
    description: str = "multiply two numbers"
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a:int, b:int) -> int:
        # synchronous implementation: runs like normal python, use when you don't need to wait for anything
        return a*b
    


In [21]:
multiply_tool1 = MultiplyTool()

In [22]:

result = multiply_tool1.invoke({"a":3, "b":2})
print(result)

6
